In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Modelo de entrenamiento

In [ ]:


from tensorflow import keras
from tensorflow.keras import layers
import os
import librosa
import numpy as np
from librosa.util import normalize
from sklearn.model_selection import train_test_split

# Definir listas para almacenar espectrogramas y etiquetas
spectrogram_list = []
labels_list = []

# Carpetas de palabras (entrenamiento)
palabra1_folder = '/content/drive/MyDrive/turnoff'
palabra2_folder = '/content/drive/MyDrive/ turn on'

# Definir etiquetas para las palabras
labels_mapping = {
    'palabra1': 0,
    'palabra2': 1,
}

# Función para cargar espectrogramas y etiquetas
def load_spectrograms_and_labels(folder, label):
    spectrogram_list = []
    labels_list = []
    for filename in os.listdir(folder):
        if filename.endswith('.wav'):
            filepath = os.path.join(folder, filename)
            audio, _ = librosa.load(filepath, sr=None)
            spectrogram = librosa.feature.melspectrogram(y=audio, sr=44100)
            spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
            if spectrogram.shape[1] < 128:
                pad_width = 128 - spectrogram.shape[1]
                spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
            else:
                spectrogram = spectrogram[:, :128]
            spectrogram_list.append(spectrogram)
            labels_list.append(label)
    return spectrogram_list, labels_list

# Cargar espectrogramas y etiquetas para cada palabra
spectrogram_list_p1, labels_list_p1 = load_spectrograms_and_labels(palabra1_folder, labels_mapping['palabra1'])
spectrogram_list_p2, labels_list_p2 = load_spectrograms_and_labels(palabra2_folder, labels_mapping['palabra2'])

# Combinar las listas de espectrogramas y etiquetas
spectrogram_list = spectrogram_list_p1 + spectrogram_list_p2
labels_list = labels_list_p1 + labels_list_p2

# Convierte las listas de espectrogramas y etiquetas en matrices de datos
X = np.array(spectrogram_list)
y = np.array(labels_list)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Añadir una dimensión adicional para el canal
X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]

# Define el modelo de aprendizaje automático
model = keras.Sequential([
    layers.Input(shape=(128, 128, 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(2, activation='softmax')  # Cambiado a 2 clases
])

# Compila el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(X_train, y_train, epochs=15)

# Evalúa el modelo
train_score = model.evaluate(X_train, y_train, verbose=0)
print('Evaluación en datos de entrenamiento:', train_score)

# Carpeta de palabras de prueba
palabra1_test_folder = '/content/drive/MyDrive/turnofftest'
palabra2_test_folder = '/content/drive/MyDrive/turnontest'

# Definir listas para almacenar espectrogramas y etiquetas de prueba
spectrogram_test_list = []
labels_test_list = []

# Cargar espectrogramas y etiquetas para cada palabra de prueba
spectrogram_test_list_p1, labels_test_list_p1 = load_spectrograms_and_labels(palabra1_test_folder, labels_mapping['palabra1'])
spectrogram_test_list_p2, labels_test_list_p2 = load_spectrograms_and_labels(palabra2_test_folder, labels_mapping['palabra2'])

# Combinar las listas de espectrogramas y etiquetas de prueba
spectrogram_test_list = spectrogram_test_list_p1 + spectrogram_test_list_p2
labels_test_list = labels_test_list_p1 + labels_test_list_p2

# Convierte las listas de prueba en matrices de datos
X_test = np.array(spectrogram_test_list)
y_test = np.array(labels_test_list)

# Añadir una dimensión adicional para el canal
X_test = X_test[:, :, :, np.newaxis]

# Evaluar el modelo en los datos de prueba
test_score = model.evaluate(X_test, y_test, verbose=0)
print('Evaluación en datos de prueba:', test_score)

# Guarda el modelo
model.save('/content/drive/MyDrive/tu_modelo.h6')



Epoch 1/15
3/3 [==============================] - 2s 264ms/step - loss: 288.6753 - accuracy: 0.4783
Epoch 2/15
3/3 [==============================] - 1s 237ms/step - loss: 150.6629 - accuracy: 0.4348
Epoch 3/15
3/3 [==============================] - 1s 256ms/step - loss: 167.2728 - accuracy: 0.4928
Epoch 4/15
3/3 [==============================] - 1s 169ms/step - loss: 121.1424 - accuracy: 0.5652
Epoch 5/15
3/3 [==============================] - 1s 146ms/step - loss: 150.2938 - accuracy: 0.5217
Epoch 6/15
3/3 [==============================] - 1s 137ms/step - loss: 144.2567 - accuracy: 0.5362
Epoch 7/15
3/3 [==============================] - 1s 143ms/step - loss: 73.5683 - accuracy: 0.4928
Epoch 8/15
3/3 [==============================] - 1s 135ms/step - loss: 89.4526 - accuracy: 0.5652
Epoch 9/15
3/3 [==============================] - 1s 140ms/step - loss: 103.6186 - accuracy: 0.5652
Epoch 10/15
3/3 [==============================] - 0s 131ms/step - loss: 58.2449 - accuracy: 0.6522
Ep

Carga de el modelo estadistico ''modelo.h''

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/tu_modelo.h6')

In [ ]:
from tensorflow import keras

# Cargar el modelo guardado
model = keras.models.load_model('/content/drive/MyDrive/tu_modelo.h6')

# Hacer predicciones con el modelo
# Puedes usar este modelo para hacer predicciones en nuevos datos de audio de la misma manera que lo hiciste en el código original.

# Ejemplo de cómo hacer una predicción en un espectrograma (X_new es tu nuevo espectrograma)
# prediction = model.predict(X_new)

# Puedes ajustar 'X_new' según tus necesidades y datos de entrada.

# Si deseas visualizar la arquitectura del modelo, puedes usar la función 'summary':
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 2)                 254018    
                                                                 
Total params: 254338 (993.51 KB)
Trainable params: 254338 (993.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Prueba de modelo.h

In [ ]:
# @title
from tensorflow import keras
import librosa
import numpy as np

# Cargar el modelo
model = keras.models.load_model('/content/drive/MyDrive/tu_modelo.h6')  # Asegúrate de proporcionar la ruta correcta si el modelo no está en el mismo directorio.

# Ruta del archivo de audio que deseas clasificar
audio_file_path = '/content/drive/MyDrive/turnoff/descarga (11).wav'  # Cambia la ruta a tu archivo de audio

# Función para cargar y preprocesar un archivo de audio
def load_and_process_audio(file_path):
    audio, _ = librosa.load(file_path, sr=None)
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=44100)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    if spectrogram.shape[1] < 128:
        pad_width = 128 - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        spectrogram = spectrogram[:, :128]
    spectrogram = spectrogram[:, :, np.newaxis]
    return spectrogram

# Cargar y preprocesar el archivo de audio
spectrogram = load_and_process_audio(audio_file_path)

# Realizar la predicción
prediction = model.predict(np.array([spectrogram]))  # El modelo espera un batch de datos, por eso usamos np.array([spectrogram]).

# La salida 'prediction' contendrá las probabilidades para cada clase.
# Imprime las probabilidades para interpretación manual.
print("Probabilidades de cada clase:", prediction)

# Establecer un umbral para clasificar
threshold = 0.5

# Clasificar la predicción
if prediction[0][0] > threshold:
    print("Turn off kitchen light")
else:
    print("Turn on kitchen light")


1/1 [==============================] - 0s 351ms/step
Probabilidades de cada clase: [[1. 0.]]
Turn off kitchen light


descarga del modelo.h


In [ ]:
!echo "const unsigned char tu_modelo[] = {" > /content/model.h
!cat /content/drive/MyDrive/tu_modelo.h6 | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("/content/model.h")
print(f"El archivo de encabezado, model.h, tiene un tamaño de {model_h_size:,} bytes.")
print("\nAbra el panel lateral (actualice si es necesario). Haga doble clic en model.h para descargar el archivo.")




cat: /content/drive/MyDrive/tu_modelo.h6: Is a directory
El archivo de encabezado, model.h, tiene un tamaño de 39 bytes.

Abra el panel lateral (actualice si es necesario). Haga doble clic en model.h para descargar el archivo.


# Nueva sección